In [44]:
!pip install pyedflib

In [45]:
import os
import numpy as np
from pyedflib import highlevel

import numpy as np

def calculate_sensitivity_specificity(ground_truth, predictions):
    confusion_matrix = np.array([[0, 0], [0, 0]])

    for gt, pred in zip(ground_truth, predictions):
        if gt == 1 and pred == 1:
            confusion_matrix[0, 0] += 1  # True Positive
        elif gt == 0 and pred == 1:
            confusion_matrix[1, 0] += 1  # False Positive
        elif gt == 0 and pred == 0:
            confusion_matrix[1, 1] += 1  # True Negative
        elif gt == 1 and pred == 0:
            confusion_matrix[0, 1] += 1  # False Negative

    TP = confusion_matrix[0, 0]
    FP = confusion_matrix[1, 0]
    TN = confusion_matrix[1, 1]
    FN = confusion_matrix[0, 1]

    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)

    return sensitivity, specificity

def file_read(folder_path, base_file_name):
    edf_files = [file for file in os.listdir(folder_path) if file.startswith(base_file_name) and file.endswith('.edf')]
    s_ws_raw = []

    for file_name in edf_files:
        file_path = os.path.join(folder_path, file_name)
        signals, signal_headers, header = highlevel.read_edf(file_path)
        s_ws_raw.append(signals)

    return s_ws_raw

def epoch_selection_ws(start_points, s_ws_raw):
    l = len(start_points)
    s_ws = np.empty((l, 34), dtype=object)

    for j in range(l):
        for i in range(34):
            s = i * 16
            e = (i - 1) * 16
            start_time = start_points[j] - s
            end_time = start_points[j] - e
            time_mask = (s_ws_raw[j][:, 1] >= start_time) & (s_ws_raw[j][:, 1] <= end_time)
            s_ws[j, i] = s_ws_raw[j][:, time_mask]

    return np.flip(s_ws, axis=1)

import numpy as np
from scipy.io import loadmat
from datetime import timedelta

def minute_selection_wos(l, s_wos_raw):
    timetable_length = s_wos_raw[0].shape[0]
    first_possible_start = 540

    lower_bound = first_possible_start
    upper_bound = timetable_length

    random_numbers = np.random.permutation(upper_bound - lower_bound + 1)[:l] + lower_bound - 1

    return random_numbers

def epoch_selection_wos(start_points, s_wos_raw):
    l = len(start_points)
    s_wos = np.empty((l, 34), dtype=object)

    for j in range(l):
        for i in range(34):
            s = i * 16
            e = (i - 1) * 16
            start_time = start_points[j] - s
            end_time = start_points[j] - e
            time_mask = (s_wos_raw[1] >= start_time) & (s_wos_raw[1] <= end_time)
            s_wos[j, i] = s_wos_raw[0][:, time_mask]

    return np.flip(s_wos, axis=1)

def feature_extraction_matrix(s_ws_total):
    l = s_ws_total.shape[0]
    array = np.zeros((l, 3910))

    for j in range(l):
        array[j, :] = feature_extraction(s_ws_total[j, :])

    return array

def feature_extraction_matrix_28(s_ws_total):
    l = s_ws_total.shape[0]
    array = np.zeros((l, 4760))

    for j in range(l):
        array[j, :] = feature_extraction(s_ws_total[j, :])

    return array

import numpy as np
from scipy.stats import entropy

def feature_extraction(s_ws):
    features = np.zeros(3910)
    t = 0

    for j in range(34):
        a = s_ws
        b = a[0, j]
        c = b[0]

        for i in range(23):
            d = c[:, 0]
            concatenated_array = np.vstack(d)

            t = t + 1
            shannon = entropy(concatenated_array)
            features[t] = shannon

            t = t + 1
            average = np.mean(concatenated_array)
            features[t] = average

            t = t + 1
            min_value = np.min(concatenated_array)
            features[t] = min_value

            t = t + 1
            max_value = np.max(concatenated_array)
            features[t] = max_value

            t = t + 1
            std_dev = np.std(concatenated_array)
            features[t] = std_dev

    return features

def feature_extraction_28(s_ws):
    features = np.zeros(4760)
    t = 0

    for j in range(34):
        a = s_ws
        b = a[0, j]
        c = b[0]

        for i in range(23):
            d = c[:, 0]
            concatenated_array = np.vstack(d)

            t = t + 1
            shannon = entropy(concatenated_array)
            features[t] = shannon

            t = t + 1
            average = np.mean(concatenated_array)
            features[t] = average

            t = t + 1
            min_value = np.min(concatenated_array)
            features[t] = min_value

            t = t + 1
            max_value = np.max(concatenated_array)
            features[t] = max_value

            t = t + 1
            std_dev = np.std(concatenated_array)
            features[t] = std_dev

    return features

In [46]:
!pip install numpy scipy

In [47]:
import numpy as np
from scipy.stats import ttest_ind
from scipy import signal

def shannon_entropy(concatenated_array):
    fs = 256
    f , Pxx= signal.periodogram(concatenated_array, fs=fs)
    norm_Pxx = Pxx / np.max(Pxx)
    entropy = -np.sum(norm_Pxx * np.log2(norm_Pxx))
    return entropy

def feature_selection(features):
    null_mean = np.zeros((1, features.shape[1]))
    p_values = np.zeros((1, features.shape[1]))

    for i in range(features.shape[1]):
        _, p_values[0, i] = ttest_ind(features[:, i], null_mean[0, i])

    alpha = 0.001
    significant_features = np.where(p_values < alpha)[1]
    significant_data = features[:, significant_features]
    return significant_data

def feature_selection_indexes(features):
    null_mean = np.zeros((1, features.shape[1]))
    p_values = np.zeros((1, features.shape[1]))

    for i in range(features.shape[1]):
        _, p_values[0, i] = ttest_ind(features[:, i], null_mean[0, i])

    alpha = 0.001
    significant_features = np.where(p_values < alpha)[1]
    return significant_features

def train_test_split(s_ws, s_wos):
    l = s_ws.shape[0]
    test_indices_negatives = np.random.choice(l, 1)
    test_indices_positives = np.random.choice(l, 1)

    test_set_negatives = s_wos[test_indices_negatives, :]
    test_set_positives = s_ws[test_indices_positives, :]

    training_set_negatives = np.delete(s_wos, test_indices_negatives, axis=0)
    training_set_positives = np.delete(s_ws, test_indices_positives, axis=0)

    return test_set_negatives, test_set_positives, training_set_negatives, training_set_positives

def feature_selection_total(features_ws, features_wos):
    combined_array = np.vstack((features_ws, features_wos))
    null_mean = np.zeros((1, combined_array.shape[1]))
    p_values = np.zeros((1, combined_array.shape[1]))

    for i in range(combined_array.shape[1]):
        _, p_values[0, i] = ttest_ind(combined_array[:, i], null_mean[0, i])

    alpha = 0.001
    significant_features = np.where(p_values < alpha)[1]
    output_variable_ws = features_ws[:, significant_features]
    output_variable_wos = features_wos[:, significant_features]
    indexes = significant_features

    return output_variable_ws, output_variable_wos, indexes

In [48]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Sub1
folder_path_ws = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub1/WS1'
base_file_name = 'chb01_'
s_ws_raw = file_read(folder_path_ws, base_file_name)
s_ws_start_points = [2296, 1467, 1732, 1015, 1720, 1862]
print(np.array(s_ws_raw[0]).shape)
s_ws = epoch_selection_ws(s_ws_start_points, s_ws_raw)

folder_path_wos = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub1/WOS1'
s_wos_raw = file_read(folder_path_wos, base_file_name)
s_wos = epoch_selection_wos(minute_selection_wos(len(s_ws_start_points), s_wos_raw), s_wos_raw)

s_ws_feature_extraction = feature_extraction_matrix(s_ws)
s_wos_feature_extraction = feature_extraction_matrix(s_wos)

# Splitting data into training and testing sets
s_ws_train_feat_ex, s_ws_test_feat_ex, s_wos_train_feat_ex, s_wos_test_feat_ex = train_test_split_custom(
    s_ws_feature_extraction, s_wos_feature_extraction, 0.5
)

# Feature selection
s_ws_train_features, s_wos_train_features, indexes_s = feature_selection_total(s_ws_train_feat_ex, s_wos_train_feat_ex)
s_ws_test_features = s_ws_test_feat_ex[:, indexes_s]
s_wos_test_features = s_wos_test_feat_ex[:, indexes_s]

# Creating labels
s_ws_train_labels = np.ones(4)
s_wos_train_labels = np.zeros(4)
s_ws_test_labels = np.ones(2)
s_wos_test_labels = np.zeros(2)

# Combining training data
s_train = np.vstack((s_ws_train_features, s_wos_train_features))
t_train = np.concatenate((s_ws_train_labels, s_wos_train_labels))

# Combining testing data
s_test = np.vstack((s_ws_test_features, s_wos_test_features))
t_test = np.concatenate((s_ws_test_labels, s_wos_test_labels))

# K-Nearest Neighbors (KNN) Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=2)
knn_classifier.fit(s_train, t_train)
predictions_knn = knn_classifier.predict(s_test)
accuracy_knn = accuracy_score(t_test, predictions_knn)
sensitivity_knn, specificity_knn = calculate_sensitivity_specificity(t_test, predictions_knn)

# Support Vector Machine (SVM) Classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(s_train, t_train)
predictions_svm = svm_classifier.predict(s_test)
accuracy_svm = accuracy_score(t_test, predictions_svm)
sensitivity_svm, specificity_svm = calculate_sensitivity_specificity(t_test, predictions_svm)

# K-Nearest Neighbors (KNN) Classifier
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {accuracy_knn}")
print(f"Sensitivity: {sensitivity_knn}")
print(f"Specificity: {specificity_knn}")
print()

# Support Vector Machine (SVM) Classifier
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm}")
print(f"Sensitivity: {sensitivity_svm}")
print(f"Specificity: {specificity_svm}")


K-Nearest Neighbors (KNN) Classifier:
Accuracy: 75
Sensitivity: 50
Specificity: 100

Support Vector Machine (SVM) Classifier:
Accuracy: 75
Sensitivity: 50
Specificity: 100


In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
folderPath_ws = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub3/WS3'
baseFileName = 'chb03_'
s_ws_raw = file_read(folderPath_ws, baseFileName)
s_ws_start_points = [731, 2162, 1982, 2592, 1725]
s_ws = epoch_selection_ws(s_ws_start_points, s_ws_raw)

folderPath_wos = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub3/WoS3'
s_wos_raw = file_read(folderPath_wos, baseFileName)
s_wos = epoch_selection_wos(minute_selection_wos(len(s_ws_start_points), s_wos_raw), s_wos_raw)

s_ws_feature_extraction = feature_extraction_matrix(s_ws)
s_wos_feature_extraction = feature_extraction_matrix(s_wos)

s_ws_train_feat_ex, s_wos_test_feat_ex, s_ws_train_labels, s_wos_test_labels = train_test_split(
    s_ws_feature_extraction, np.ones(3), s_wos_feature_extraction, np.zeros(2), test_size=0.5
)

s_ws_train_features, s_wos_train_features, indexes_s = feature_selection_total(
    s_ws_train_feat_ex, s_wos_train_feat_ex
)
s_ws_test_features = s_ws_test_feat_ex[:, indexes_s]
s_wos_test_features = s_wos_test_feat_ex[:, indexes_s]

s = np.vstack((s_ws_train_features, s_wos_train_features))
t = np.concatenate((s_ws_train_labels, s_wos_train_labels))

s_test = np.vstack((s_ws_test_features, s_wos_test_features))
t_test = np.concatenate((s_ws_test_labels, s_wos_test_labels))

knnClassifier = KNeighborsClassifier(n_neighbors=2)
knnClassifier.fit(s, t)
predictions_knn = knnClassifier.predict(s_test)
accuracy_knn = accuracy_score(t_test, predictions_knn)
sensitivity_knn, specificity_knn = calculateSensitivitySpecificity(t_test, predictions_knn)

svmClassifier = SVC(kernel='linear')
svmClassifier.fit(s, t)
predictions_svm = svmClassifier.predict(s_test)
accuracy_svm = accuracy_score(t_test, predictions_svm)
sensitivity_svm, specificity_svm = calculateSensitivitySpecificity(t_test, predictions_svm)

# K-Nearest Neighbors (KNN) Classifier
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {accuracy_knn}")
print(f"Sensitivity: {sensitivity_knn}")
print(f"Specificity: {specificity_knn}")
print()

# Support Vector Machine (SVM) Classifier
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm}")
print(f"Sensitivity: {sensitivity_svm}")
print(f"Specificity: {specificity_svm}")


K-Nearest Neighbors (KNN) Classifier:
Accuracy: 100
Sensitivity: 100
Specificity: 100

Support Vector Machine (SVM) Classifier:
Accuracy: 75
Sensitivity: 100
Specificity: 50


In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Sub8
folderPath_ws = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub8/WS8'
baseFileName = 'chb08_'
s_ws_raw = file_read(folderPath_ws, baseFileName)
s_ws_start_points = [2670, 2856, 2988, 2417, 2083]
s_ws = epoch_selection_ws(s_ws_start_points, s_ws_raw)

folderPath_wos = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub8/WOS8'
s_wos_raw = file_read(folderPath_wos, baseFileName)
s_wos = epoch_selection_wos(minute_selection_wos(len(s_ws_start_points), s_wos_raw), s_wos_raw)

s_ws_feature_extraction = feature_extraction_matrix(s_ws)
s_wos_feature_extraction = feature_extraction_matrix(s_wos)

s_wos_test_feat_ex, s_ws_test_feat_ex, s_wos_train_feat_ex, s_ws_train_feat_ex = train_test_split(
    s_ws_feature_extraction, np.ones(3), s_wos_feature_extraction, np.zeros(2), test_size=0.5
)

s_ws_train_features, s_wos_train_features, indexes_s = feature_selection_total(
    s_ws_train_feat_ex, s_wos_train_feat_ex
)
s_ws_test_features = s_ws_test_feat_ex[:, indexes_s]
s_wos_test_features = s_wos_test_feat_ex[:, indexes_s]

s_ws_train_labels = np.ones(3)
s_wos_train_labels = np.zeros(3)
s_ws_test_labels = np.ones(2)
s_wos_test_labels = np.zeros(2)

s = np.vstack((s_ws_train_features, s_wos_train_features))
t = np.concatenate((s_ws_train_labels, s_wos_train_labels))

s_test = np.vstack((s_ws_test_features, s_wos_test_features))
t_test = np.concatenate((s_ws_test_labels, s_wos_test_labels))

knnClassifier = KNeighborsClassifier(n_neighbors=2)
knnClassifier.fit(s, t)
predictions_knn = knnClassifier.predict(s_test)
accuracy_knn = accuracy_score(t_test, predictions_knn)
sensitivity_knn, specificity_knn = calculateSensitivitySpecificity(t_test, predictions_knn)

svmClassifier = SVC(kernel='linear')
svmClassifier.fit(s, t)
predictions_svm = svmClassifier.predict(s_test)
accuracy_svm = accuracy_score(t_test, predictions_svm)
sensitivity_svm, specificity_svm = calculateSensitivitySpecificity(t_test, predictions_svm)

# K-Nearest Neighbors (KNN) Classifier
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {accuracy_knn}")
print(f"Sensitivity: {sensitivity_knn}")
print(f"Specificity: {specificity_knn}")
print()

# Support Vector Machine (SVM) Classifier
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm}")
print(f"Sensitivity: {sensitivity_svm}")
print(f"Specificity: {specificity_svm}")

K-Nearest Neighbors (KNN) Classifier:
Accuracy: 100
Sensitivity: 100
Specificity: 100

Support Vector Machine (SVM) Classifier:
Accuracy: 100
Sensitivity: 100
Specificity: 100


In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Function definitions for file_read, epoch_selection_ws, epoch_selection_wos,
# minute_selection_wos, feature_extraction_matrix, train_test_split,
# feature_selection_total, calculateSensitivitySpecificity should be provided separately.

# Define file_read, epoch_selection_ws, epoch_selection_wos, minute_selection_wos,
# feature_extraction_matrix, train_test_split, feature_selection_total,
# calculateSensitivitySpecificity functions here.

# Sub14
folderPath_ws = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub14/WS14'
baseFileName = 'chb14_'
s_ws_raw = file_read(folderPath_ws, baseFileName)
s_ws_start_points = [1986, 1911, 1838, 3239, 1039, 2833]
s_ws = epoch_selection_ws(s_ws_start_points, s_ws_raw)

folderPath_wos = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub14/WOS14'
s_wos_raw = file_read(folderPath_wos, baseFileName)
s_wos = epoch_selection_wos(minute_selection_wos(len(s_ws_start_points), s_wos_raw), s_wos_raw)

s_ws_feature_extraction = feature_extraction_matrix(s_ws)
s_wos_feature_extraction = feature_extraction_matrix(s_wos)

s_wos_test_feat_ex, s_ws_test_feat_ex, s_wos_train_feat_ex, s_ws_train_feat_ex = train_test_split(
    s_ws_feature_extraction, np.ones(4), s_wos_feature_extraction, np.zeros(2), test_size=0.5
)

s_ws_train_features, s_wos_train_features, indexes_s = feature_selection_total(
    s_ws_train_feat_ex, s_wos_train_feat_ex
)
s_ws_test_features = s_ws_test_feat_ex[:, indexes_s]
s_wos_test_features = s_wos_test_feat_ex[:, indexes_s]

s_ws_train_labels = np.ones(4)
s_wos_train_labels = np.zeros(4)
s_ws_test_labels = np.ones(2)
s_wos_test_labels = np.zeros(2)

s = np.vstack((s_ws_train_features, s_wos_train_features))
t = np.concatenate((s_ws_train_labels, s_wos_train_labels))

s_test = np.vstack((s_ws_test_features, s_wos_test_features))
t_test = np.concatenate((s_ws_test_labels, s_wos_test_labels))

knnClassifier = KNeighborsClassifier(n_neighbors=2)
knnClassifier.fit(s, t)
predictions_knn = knnClassifier.predict(s_test)
accuracy_knn = accuracy_score(t_test, predictions_knn)
sensitivity_knn, specificity_knn = calculateSensitivitySpecificity(t_test, predictions_knn)

svmClassifier = SVC(kernel='linear')
svmClassifier.fit(s, t)
predictions_svm = svmClassifier.predict(s_test)
accuracy_svm = accuracy_score(t_test, predictions_svm)
sensitivity_svm, specificity_svm = calculateSensitivitySpecificity(t_test, predictions_svm)
# K-Nearest Neighbors (KNN) Classifier
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {accuracy_knn}")
print(f"Sensitivity: {sensitivity_knn}")
print(f"Specificity: {specificity_knn}")
print()

# Support Vector Machine (SVM) Classifier
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm}")
print(f"Sensitivity: {sensitivity_svm}")
print(f"Specificity: {specificity_svm}")

K-Nearest Neighbors (KNN) Classifier:
Accuracy: 100
Sensitivity: 100
Specificity: 100

Support Vector Machine (SVM) Classifier:
Accuracy: 100
Sensitivity: 100
Specificity: 100


In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Sub16
folderPath_ws = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub16/WS16'
baseFileName = 'chb16_'
s_ws_raw = file_read(folderPath_ws, baseFileName)
s_ws_start_points = [2290, 1120, 1854, 1214]
s_ws = epoch_selection_ws(s_ws_start_points, s_ws_raw)

folderPath_wos = '/content/drive/MyDrive/Colab Notebooks/CHB-MIT Scalp EEG Database/Sub16/WOS16'
s_wos_raw = file_read(folderPath_wos, baseFileName)
s_wos = epoch_selection_wos(minute_selection_wos(len(s_ws_start_points), s_wos_raw), s_wos_raw)

s_ws_feature_extraction = feature_extraction_matrix(s_ws)
s_wos_feature_extraction = feature_extraction_matrix(s_wos)

s_wos_test_feat_ex, s_ws_test_feat_ex, s_wos_train_feat_ex, s_ws_train_feat_ex = train_test_split(
    s_ws_feature_extraction, np.ones(3), s_wos_feature_extraction, np.zeros(1), test_size=0.5
)

s_ws_train_features, s_wos_train_features, indexes_s = feature_selection_total(
    s_ws_train_feat_ex, s_wos_train_feat_ex
)
s_ws_test_features = s_ws_test_feat_ex[:, indexes_s]
s_wos_test_features = s_wos_test_feat_ex[:, indexes_s]

s_ws_train_labels = np.ones(3)
s_wos_train_labels = np.zeros(3)
s_ws_test_labels = np.ones(1)
s_wos_test_labels = np.zeros(1)

s = np.vstack((s_ws_train_features, s_wos_train_features))
t = np.concatenate((s_ws_train_labels, s_wos_train_labels))

s_test = np.vstack((s_ws_test_features, s_wos_test_features))
t_test = np.concatenate((s_ws_test_labels, s_wos_test_labels))

knnClassifier = KNeighborsClassifier(n_neighbors=2)
knnClassifier.fit(s, t)
predictions_knn = knnClassifier.predict(s_test)
accuracy_knn = accuracy_score(t_test, predictions_knn)
sensitivity_knn, specificity_knn = calculateSensitivitySpecificity(t_test, predictions_knn)

svmClassifier = SVC(kernel='linear')
svmClassifier.fit(s, t)
predictions_svm = svmClassifier.predict(s_test)
accuracy_svm = accuracy_score(t_test, predictions_svm)
sensitivity_svm, specificity_svm = calculateSensitivitySpecificity(t_test, predictions_svm)

# K-Nearest Neighbors (KNN) Classifier
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {accuracy_knn}")
print(f"Sensitivity: {sensitivity_knn}")
print(f"Specificity: {specificity_knn}")
print()

# Support Vector Machine (SVM) Classifier
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm}")
print(f"Sensitivity: {sensitivity_svm}")
print(f"Specificity: {specificity_svm}")

K-Nearest Neighbors (KNN) Classifier:
Accuracy: 75
Sensitivity: 100
Specificity: 50

Support Vector Machine (SVM) Classifier:
Accuracy: 75
Sensitivity: 100
Specificity: 50
